In [21]:
DB_HOST = "localhost"
DB_PORT = "3306"
DB_USER = "root"
DB_PASSWORD = "root"
DATABASE = "ALGAS"

CREATE_QUERY = "CREATE TABLE IF NOT EXISTS dados (id INT PRIMARY KEY AUTO_INCREMENT, valor INT, espacoMemoria INT, tempoExecucao INT)"
INSERT_QUERY = "INSERT INTO dados (id, valor, espacoMemoria, tempoExecucao) VALUES (default, %s, %s, %s)"
SELECT_QUERY = "SELECT id, valor, espacoMemoria, tempoExecucao FROM dados"
DELETE_QUERY = "TRUNCATE dados"

DROP_QUERY = "DROP TABLE dados"

In [47]:
from mysql.connector import connect as connect_my
def get_connection():
    return connect_my(
        # host = DB_HOST + ":" + DB_PORT,
        host = DB_HOST,
        user = DB_USER,
        password = DB_PASSWORD,
        database = DATABASE
        )

def create_db():
    con = get_connection()
    c = con.cursor()
    c.execute(CREATE_QUERY)    
    return con

In [59]:
def get_insert_query():
    return INSERT_QUERY

def insert_data(self, data: list):
    con = get_connection()
    c = con.cursor()
    
    insert = get_insert_query()

    for value in data:
        c.execute(insert, value)

    con.commit()

def select_data():
    con = get_connection()
    c = con.cursor()
    
    c.execute(SELECT_QUERY)
    
    return c.fetchall()

In [53]:
class DatabaseManager:
    def __init__(self):
        self.connection = self.create()
        self.cursor = None
    
    def create(self):
        return get_connection()
    
    def transact_begin(self):
        self.cursor = self.connection.cursor()
    
    def transact_commit(self):
        self.connection.commit()

    def insert(self, dados: list):
        insert_data(self.connection, dados)
        
    def select(self) -> list:
        return select_data(cursor=self.connection.cursor())


In [24]:
from datetime import datetime
from sys import getsizeof


def get_data_ranges() -> tuple:
    return (
        range(100000, 600000, 100000),
        range(1000, 6000, 100), 
        range(100, 600, 100), 
        range(10, 60, 10), 
        range(1000000, 6000000, 1000000)
    )


def append_data(data: list) -> list:
    dados = []
    memory_accumulator = 0
    for range_data in data:
        for value in range_data:
            inicio = datetime.now()
            acumulador = 0
            for i in range(value, 0, -1):
                acumulador += i
            memory_accumulator += getsizeof(value)
            duracao = datetime.now() - inicio
            dados.append((value, memory_accumulator, duracao.microseconds))
    return dados


def generate_data(sort: bool = False) -> list:
    dados = append_data(get_data_ranges())

    if sort:
        dados = sorted(dados, key=lambda x: x[0])

    return dados 

In [60]:
dadosX = generate_data()
db = DatabaseManager()
db.transact_begin()
#for dado in dadosX:
db.insert(dadosX)

db.transact_commit()
print("Inseriu")

Inseriu
